In [132]:
import pandas as pd, numpy as np
from influxdb import DataFrameClient
import json
# import feedparser

Set params

In [133]:
purge=False
# purge=True

In [134]:
process_annoations=False
# process_annoations=True

In [135]:
# process_macro=False
process_macro=True

In [136]:
# write_out=False
write_out=True

In [137]:
full_overwrite=False
# full_overwrite=True

Load pre-params

In [138]:
df0=pd.read_csv('df0.csv').set_index('date')
df0.index=pd.to_datetime(df0.index)

In [139]:
bets=json.loads(open('bets.json','r').read())

In [140]:
severity=pd.read_csv('severity.csv').set_index('date')
severity.index=pd.to_datetime(severity.index)

In [141]:
mobility=pd.read_csv('mobility.csv').set_index('date')
mobility.index=pd.to_datetime(mobility.index)

In [142]:
mobility_mini=pd.read_csv('mobility_mini.csv').set_index('date')
mobility_mini.index=pd.to_datetime(mobility_mini.index)

Establish DB connection

In [143]:
user = 'admin'
password = open('auth/influxa.txt','r').read()
host='influxdb'
port=8086
dbname='base'
dbname_long='long'
protocol = 'line' #'json'
client = DataFrameClient(host, port, user, password, dbname)
client_long = DataFrameClient(host, port, user, password, dbname_long)

In [144]:
if purge:
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '600d', 1, default=True)
    client_long.drop_database(dbname_long)
    client_long.drop_retention_policy(dbname_long)
    client_long.create_database(dbname_long)
    client_long.create_retention_policy(dbname_long, '6000d', 1, default=True)

In [145]:
htmlipath='../html/'
htmlepath='//myserv-html.er/'
htmlepath_other='//mybackupserv-html.er/'

In [146]:
titles={'HU':"Magyar",'RO':'Română','EN':'English'}

In [147]:
rtitles={titles[t]:t for t in titles}

In [148]:
import requests
grafana = "http://grafana:3000/"
headers = {
    'Authorization': 'Bearer '+open('auth/grafana.txt','r').read(),
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [149]:
response = requests.get(grafana+'api/folders', headers=headers)
folders=json.loads(response.content)

In [150]:
folder_id=[f['id'] for f in folders if f['title']=='My Grafana Folder'][0] #General

In [151]:
response = requests.get(grafana+'api/search?folderIds='+str(folder_id), headers=headers)
dashs=json.loads(response.content)

In [152]:
uids={rtitles[d['title']]:d['uid'] for d in dashs if d['title'] in rtitles}
iids={rtitles[d['title']]:d['id'] for d in dashs if d['title'] in rtitles}

In [153]:
uids_light={rtitles[d['title'].split(' Light')[0]]:d['uid'] for d in dashs if 'Light' in d['title']}
iids_light={rtitles[d['title'].split(' Light')[0]]:d['id'] for d in dashs if 'Light' in d['title']}

In [155]:
languages=['HU','RO','EN']

In [156]:
url='https://docs.google.com/spreadsheets/d/'+open('auth/sheet.txt','r').read()+'/gviz/tq?tqx=out:csv&sheet='

In [157]:
def get_szotar():
    sheet='szotar'
    columns=languages+[i+'_description' for i in languages]+[i+'_source' for i in languages]
    df=pd.read_csv(url+sheet)
    df=df[['ID','UI']+columns]
    sheet='minidashboard'
    df2=pd.read_csv(url+sheet)
    df2=df2[['ID','UI']+columns]
    df=pd.concat([df,df2])
    szotardf=df.set_index('ID')[columns]
    szotar=df.set_index('ID').T.to_dict()
    szotarHU=df.set_index('HU',drop=False).T.to_dict()
    szotarRO=df.set_index('RO',drop=False).T.to_dict()
    szotarEN=df.set_index('EN',drop=False).T.to_dict()
    return szotardf,szotar,szotarHU,szotarRO,szotarEN

In [158]:
def resolve_time_conflicts(series):
    ds={}
    ts=[]
    for d in series:
        if d not in ds:
            ds[d]=pd.to_datetime(d)
            t=(ds[d])
        else:
            ds[d]=ds[d]+pd.to_timedelta('193m')
            t=(ds[d])
        ts.append(t)
    return ts

# Influx

In [159]:
szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

<ipython-input-157-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-157-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-157-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [160]:
import time
import datetime
import pytz
utc=pytz.UTC

In [161]:
def push2influx(df,measurement,field_columns,tag_columns,shift=False,dbclient=client,wo=write_out,fo=full_overwrite,daily=True):
    if wo:
        df=df.sort_index()
        # df.index = utc.localize(df.index) 
        df.index=df.index.tz_localize('GMT')#.tz_convert('Europe/Bucharest')
        if shift:
            df.index+=pd.to_timedelta('12h')
        if fo: 
            print('Purging',measurement,'...')
            dbclient.query('DROP MEASUREMENT '+measurement)
        else:
            latest=dbclient.query('SELECT * FROM '+measurement+' GROUP BY "1d" ORDER BY DESC LIMIT 1')
            if latest:
                lat=latest[list(latest.keys())[0]].index[0]
                if daily: lat+=pd.to_timedelta('1d')
                df=df[lat:]
                print('Slicing',measurement,'from',lat,'...')
            else:
                print('No data in db for',measurement,'...')
        time.sleep(3)
        print('Writing to',measurement,'...')
        bsize=5000
        bwait=2
        print(len(df),'data points will be written in',len(df)/bsize,'batches.')
        print('Expected query running time is:',int((len(df)/bsize)*bwait*1.1)+3,'seconds.')
        for i in range(int(len(df)/bsize)+1):
            r=range(i*bsize,min(len(df),(i+1)*bsize))
            dc=df.iloc[r]
            print('Writing batch',i+1,'...')
            dbclient.write_points(dc, measurement, protocol=protocol,
                                field_columns=field_columns,
                                tag_columns=[])
            time.sleep(bwait)
        time.sleep(3)
        print('Done!')
    else:
        print('Write-out not enabled. Skipping...')

DateLaZi

In [162]:
#new method from DateLaZi
dlz=requests.get('https://datelazi.ro/latestData.json').content
dlz=json.loads(dlz)
dlz['historicalData'][dlz['currentDayStats']['parsedOnString']]=dlz['currentDayStats']
dlzs=[]
dlz_counties=[]
for date in dlz['historicalData']:
    d={}
    mdate=date.replace('2018-11-07','2020-11-07').replace('2020-01-05','2021-01-05').replace('2020-01-07','2021-01-07')
    d['date']=mdate
    d['cases']=dlz['historicalData'][date]['numberInfected']
    d['heals']=dlz['historicalData'][date]['numberCured']
    d['deaths']=dlz['historicalData'][date]['numberDeceased']
    if 'vaccines' in dlz['historicalData'][date]:
        if dlz['historicalData'][date]['vaccines']:
            for v in dlz['historicalData'][date]['vaccines']:
                for k in ['total_administered','immunized']:
                    k2=k.replace('immunized','total_immunized')
                    if k2 not in d: d[k2]=0
                    d[k2]+=dlz['historicalData'][date]['vaccines'][v][k]
                    d[k2+'_'+v]=dlz['historicalData'][date]['vaccines'][v][k]
    if 'countyInfectionsNumbers' in dlz['historicalData'][date]:
        counties=dlz['historicalData'][date]['countyInfectionsNumbers']
        if counties:
            for county in counties:
                dummy={'date':mdate,county:dlz['historicalData'][date]['countyInfectionsNumbers'][county]}
                if county=='PH':
                    if date=='2020-12-01':
                        dummy[county]=19395
                dlz_counties.append(dummy)
    dlzs.append(d)
dl=pd.DataFrame(dlzs).set_index('date').sort_index(ascending=True)

In [163]:
#Manual fixes
dl.loc['2021-04-08','cases']=993613
dl.loc['2021-04-08','heals']=890528
dl.loc['2021-04-08','deaths']=24733

In [164]:
for c in dl.columns[3:]:
    dl[c]=dl[c].cumsum()
dl2=dl.diff()
dl2.columns=['case','heal','death']+[i.replace('total_','') for i in dl2.columns[3:]]
dl['active']=dl['cases']-dl['heals']-dl['deaths']
dl=dl.join(dl2)

In [165]:
latest=dl.index[-1]
now=str(pd.to_datetime("now"))[:10]
print(latest,now)

2021-04-08 2021-04-09


Tests from `graphs.ro`

In [166]:
r=requests.get('https://www.graphs.ro/json.php')
graphs_ro=json.loads(r.content)['covid_romania']
g=pd.DataFrame(graphs_ro)
# g['date']=pd.to_datetime(g['reporting_date'])
g['date']=g['reporting_date']
g=g.set_index('date')
g['tests']=g['total_tests']
g['test']=g['new_tests_today']
latest2=g.index[0]
print(latest2)

2021-04-08


In [167]:
latest=str(min(pd.to_datetime(latest),pd.to_datetime(latest2)))[:10]
print(latest)

2021-04-08


In [168]:
dl=dl.join(g[['tests','test']])

Governance

In [206]:
sheet='governance'

In [207]:
df=pd.read_csv(url+sheet)
# dl=pd.concat([df.set_index('date').sort_index()[:'2020-03-17'][dl.columns[:-1]],
#           dl['2020-03-18':]])
df=df.set_index('date').sort_index(ascending='True').join(dl,lsuffix='_g')#[20:] #[:-1] vaccines nincs a governance-ban

In [208]:
# fill from local, if missing in DateLaZi
for c in ['active','cases','heals','deaths','case','heal','death']:
    df[c]=df[c].fillna(df[c+'_g'].astype(str).str.replace(',','')).replace('nan',np.nan)
df=df.dropna(how='all',axis=0).dropna(how='all',axis=1)

In [209]:
df14=df[['cases']].reset_index().join(df[['cases']][14:].reset_index(),lsuffix='1')
df14['case14']=df14['cases'].astype(float)-df14['cases1'].astype(float)
df=df.join(df14.set_index('date')['case14'].dropna().astype(int))

In [210]:
df.index=pd.to_datetime(df.index)
df0=df.copy()
#df['date']=pd.to_datetime(df['date'])
#df=df[df.columns[:13]].set_index('date')
vaccine_totals=[i for i in list(dl2.columns[3:])+list(dl.columns[3:len(dl2.columns)]) if 'total_' in i]
df=pd.DataFrame(df[['active','cases','heals','deaths','case14']+\
                  vaccine_totals].stack()).reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float).astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']
current=df[df['type']=='case14']['value'][-1] #sajat becsles
# current=df[df['type']=='active']['value'][-1] #official
current

73819

In [211]:
df0.to_csv('df0.csv')

In [215]:
#sanity check
cols=[ 'active','case14', 'case', 'heal', 'death','cases', 'heals', 'deaths']
dx=df0[cols].dropna(how='all')
dx=dx.loc[~dx.index.isnull()]
dx[cols].tail()

,active,case14,case,heal,death,cases,heals,deaths
date,,,,,,,,
2021-04-04,78355.0,77260.0,4151.0,4287.0,97.0,974375.0,871950.0,24070.0
2021-04-05,78309.0,77128.0,3611.0,3537.0,120.0,977986.0,875487.0,24190.0
2021-04-06,78402.0,76210.0,5231.0,4942.0,196.0,983217.0,880429.0,24386.0
2021-04-07,78353.0,75481.0,5407.0,5281.0,175.0,988624.0,885710.0,24561.0
2021-04-08,78352.0,73819.0,4989.0,4818.0,172.0,993613.0,890528.0,24733.0


-> Run till here to test Grafana only.

In [44]:
df1=df.replace(0,np.nan).dropna()
df1=df1[df1['type'].isin(['active', 'cases', 'heals', 'deaths', 'case14'])]
df2=df.replace(0,np.nan).dropna()
df2=df2[~(df2['type'].isin(['active', 'cases', 'heals', 'deaths', 'case14']))]

In [45]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='governance1'
# push2influx(df1,measurement,field_columns,tag_columns,fo=True)
push2influx(df1,measurement,field_columns,tag_columns)
measurement='vaccine1'
# push2influx(df2,measurement,field_columns,tag_columns,fo=True)
push2influx(df2,measurement,field_columns,tag_columns)

Slicing governance1 from 2021-04-09 00:00:00+00:00 ...
Writing to governance1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!
Slicing vaccine1 from 2021-04-09 00:00:00+00:00 ...
Writing to vaccine1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [46]:
# df=pd.read_csv(url+sheet)
df=df0.copy()

# df['date']=pd.to_datetime(df['date'])
# df=df[df.columns[:13]].set_index('date')

Daily cases

In [47]:
df['case/test']=100*df['case'].astype(str).str.replace(',','').astype(float)/\
    df['test'].astype(str).str.replace(',','').astype(float)
df['death_rate']=100*df['death'].astype(str).str.replace(',','').astype(float)/\
    df['case'].astype(str).str.replace(',','').astype(float)

In [48]:
df=pd.DataFrame(df[['death','heal','case','case/test','death_rate']].stack()).reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']

In [49]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='governance2'
# push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns,fo=True)
push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns)

Slicing governance2 from 2021-04-09 00:00:00+00:00 ...
Writing to governance2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Small dashboard

In [51]:
sheet='minidashboard'
dm=pd.read_csv(url+sheet)
df=df0[list(dl.columns)].dropna(how='all')
df=df.loc[~df.index.isnull()]
for i in dm.set_index('ID')['UI'].iteritems():
    df[i[0]]=i[1]
sheet='EcMonitor'
du=pd.read_csv(url+sheet)
du=du.set_index('date').loc['Unemployment ratio'][2:].dropna()
du.index=[(pd.to_datetime('now')-pd.to_timedelta('11D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(du))][::-1]
du=pd.DataFrame(du)
du.columns=['unemployment']
du.index=pd.to_datetime(du.index)
df=df.join(du)
sheet='employmentdata'
de=pd.read_csv(url+sheet)
de['date']=pd.to_datetime(de['date'])
de=de[de.columns[:3]].set_index('date')
de.columns=['felbontott','felfuggesztett']
# df=df.join(de.ffill())
for c in de.columns:
    de[c]=de[c].astype(str).str.replace(',','').str.replace('%','').astype(float)#.interpolate(method='linear').dropna()
df=df.join(de)
df=df.join(mobility_mini)
for c in df.columns:
    df[c]=df[c].astype(str).str.replace(',','').str.replace('%','').astype(float)
sheet='exchangerates'
dx=pd.read_csv(url+sheet)
dx=dx[['date','EUR - RON (megváltozás)']][5:-1]
dx['date']=pd.to_datetime(dx['date'])
dx=dx.set_index('date')
dx.columns=['xch']
dx['xch']=dx['xch'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index().cumsum()
df=df.join(dx)
df['xch']=df['xch'].ffill()
sheet='stocks_all'
dk=pd.read_csv(url+sheet)[bets].dropna()
dk['date']=pd.to_datetime(dk[bets[0]])
dk=dk.set_index('date').drop(bets[0],axis=1)
dk.columns=['bet']
dk=dk.reindex(df.index)
dk['bet']=dk['bet'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index().cumsum()
df=df.join(dk)
sheet='MiniGDP'
dg=pd.read_csv(url+sheet)
dg1=dg[['Date','GDP (QoQ, SA)']].dropna()
dg1.index=[(pd.to_datetime('now')-pd.to_timedelta('2D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg1))][::-1]
dg2=dg[['Date.1','GDP']].dropna()
dg2.index=[(pd.to_datetime('now')-pd.to_timedelta('8D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg2))][::-1]
sheet='DFMChartYoY'
dg3=pd.read_csv(url+sheet).T
dg3=pd.read_csv(url+sheet,skiprows=19,nrows=2).T.iloc[1:].dropna()[1]
dg3.index=[(pd.to_datetime('now')-pd.to_timedelta('1D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg3))][::-1]
dg=dg1.join(dg2).join(dg3)[['GDP','GDP (QoQ, SA)',1]].astype(float)
dg.columns=['gdp','gdpq','employees']
dg.index=pd.to_datetime(dg.index)
df=df.join(dg)
df=df[:pd.to_datetime('now')]

In [52]:
field_columns=list(df.columns)
tag_columns=[]
measurement='governance3'
# push2influx(df,measurement,field_columns,tag_columns,wo=True,fo=True)
push2influx(df,measurement,field_columns,tag_columns,wo=True)

Slicing governance3 from 2021-04-09 00:00:00+00:00 ...
Writing to governance3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


# Grafana

In [53]:
today0=now[-2:]
if today0[0]=='0': 
    today=today0[1]
else:
    today=today0
currents=str(current)[:-3]+'&nbsp;'+str(current)[-3:]
hour=str(pd.to_datetime('now')+pd.to_timedelta('3h'))[11:16]
str(szotar['report']['UI'][1:]),current,today0,hour

('408', 73819, '09', '10:02')

In [54]:
def darken2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0xfefefe >> 1))[2:]
def brighten2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0x7f7f7f << 1))[2:]
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.rgb2hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [55]:
# szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

In [56]:
colors_to_darken=['#F2CC0C','#CA95E5','#FF780A','#E0B400','#96D98D','#F2495C','#E02F44',
                  'lime','#73BF69','#8AB8FF','#3274D9','#D3D3D3','#FF7383','#FADE2A']

# Maps

In [57]:
top=3

In [58]:
# iso_counties=pd.read_html('https://en.wikipedia.org/wiki/Counties_of_Romania')[1]\
#     .set_index(['ISO code[note 3]'])['County'].to_dict()
# iso_counties['BN']='Bistrița-Năsăud'
# iso_counties['CL']='Călărași'
# open(htmlipath+'panels/iso_counties.json','w').write(json.dumps(iso_counties))
iso_counties=json.loads(open(htmlipath+'panels/iso_counties.json','r').read())

In [59]:
georo=json.loads(open(htmlipath+'panels/romania-counties.json','r').read())
georoco={i['properties']['NAME_1']:i['properties']['ID_1'] for i in georo['objects']['ROU_adm1']['geometries']}   
georoco['București']=georoco['Bucharest']

In [60]:
sheet='industry_county'
df=pd.read_csv(url+sheet)[:-2]
df=df[df.columns[:5]]
df.columns=['HU','RO','EN','q2','q3']
for q in ['q2','q3']:
    df[q]=df[q].str.replace('%','').astype(float)
df=df.set_index('RO').join(pd.DataFrame(georoco,index=['county']).T).reset_index().dropna()
df.columns=['RO','HU','EN','q2','q3','county']

In [61]:
for lang in languages:
    dc=df[[lang,'q2','q3','county']]
    dc.columns=['county','q2','q3','id']
    for c in ['q2','q3']:
        dd=dc.set_index([c])[['county']].sort_index(ascending=False)
        dds=[]
        ddd=dd.iloc[:top]
        ddd[c+'_high']=True
        dds.append(ddd)
        ddd=dd.iloc[-top:]
        ddd[c+'_high']=False
        dds.append(ddd)
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['county']).join(dds.reset_index().set_index(['county'])\
                                         [[c+'_display',c+'_high']]).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        
    dcqs=[]
    for c in ['q2','q3']:
        dcq=dc.set_index([c,'county'])[[c+'_display',c+'_high']]
        dcq.index.names=['value','county']
        dcq['quarter']=c
        dcq=dcq.reset_index().set_index(['quarter','county','value'])
        dcq.columns=['display','high']
        dcq['high']=dcq['high'].fillna(False)
        dcqs.append(dcq)
    dcqs=pd.concat(dcqs)
    dcc=dc.set_index(['county','id'])[['q2','q3']].stack().reset_index()
    dcc.columns=['county','id','quarter','value']
    dcc=dcc.set_index(['quarter','county','value']).join(dcqs).reset_index()
    dcc['q']=dcc['quarter'].replace({'q2':'Q2 vs Q1','q3':'Q3 vs Q2'})
    dcc['q0']=dcc['quarter'].replace({'q2':2019,'q3':2020})
    # dcc=dcc.sort_values('quarter',ascending=False)
    open(htmlipath+'panels/county_'+lang+'.json','w').write(json.dumps(list(dcc.T.to_dict().values())))
    # open('county_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

<ipython-input-61-1a3a71a98fd4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd[c+'_high']=True
<ipython-input-61-1a3a71a98fd4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd[c+'_high']=False


Case county map

In [62]:
sheet='counties_current'
dg=pd.read_csv(url+sheet, header=None)
dgl=dg[[1,4]].set_index(4).T.to_dict()
dgl['Dambovita']=dgl['Dâmbovita']
dgl['Szucsáva']=dgl['Suceava']
dgl['Galac']=dgl['Galati']
dgl['Valcea']=dgl['Vâlcea']
dfl=df[languages+['county']].set_index('HU')
dh=dfl.join(pd.DataFrame(dgl).T).reset_index().dropna()

In [63]:
for lang in languages:
    dc=dh[[lang,1,'county']]
    dc.columns=['county','value','id']
    open(htmlipath+'panels/county2_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))
    # open('county2_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

Case county map fancy

In [64]:
pop=json.loads(open(htmlipath+'panels/pop.json').read())
pop={i:pop[i][1] for i in pop}
pdf=pd.DataFrame(pop,index=['pop']).T
pdf.index.name='iso'

In [65]:
dq=pd.DataFrame(dlz_counties).groupby('date').mean().drop('-',axis=1).stack().reset_index()
dq.columns=['date','iso','cases']
iso2=pd.DataFrame(iso_counties,index=['EN']).T.reset_index()
iso2.columns=['iso','EN']
dq=dq.set_index('iso').join(pdf).join(iso2.set_index('iso')).set_index('EN').join(dh.set_index('EN').drop(1,axis=1)).reset_index()
dq['case_cap']=dq['cases']*1000/dq['pop']

In [66]:
dmss=[]
for lang in languages:
    dms=[]
    # dm=dq.set_index([lang,'date'])[['cases','case_cap','pop','county']]
    dm=dq.set_index('EN').join(iso2.set_index('EN')).reset_index().set_index([lang,'date'])[['cases','case_cap','pop','county','iso']]
    for m in dm.index.get_level_values(0).unique():
        dc=dm.loc[m].sort_index()
        p=dc.iloc[0]['pop']
        dc['case_14']=dc['cases'].diff().rolling(14).sum()
        dc['case_14_cap']=dc['case_14']*1000/p
        dc['id']=dc['county']
        dc['county']=m
        dms.append(dc.dropna())
    dms=pd.concat(dms)
    dms=dms[((dms['case_14_cap']<14)&(dms['case_14_cap']>0))] #filter out non-sense values
    open(htmlipath+'panels/county2c_'+lang+'.json','w').write(json.dumps(list(dms.reset_index().T.to_dict().values())))
    dms2=dms.sort_index()['2020-08-05':]
    open(htmlipath+'panels/county2d_'+lang+'.json','w').write(json.dumps(list(dms2.reset_index().T.to_dict().values())))
    
    dc=dms.reset_index().sort_index(ascending=False)
    for c in ['cases','case_cap','case_14','case_14_cap']:
        dd=dc.reset_index().set_index(['date',c])[['county']].sort_index(ascending=False)
        dds=[]
        for d in dd.index.get_level_values(0).unique():
            dds.append(dd.loc[[d]].iloc[:top])
            dds.append(dd.loc[[d]].iloc[-top:])
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['date','county']).join(dds.reset_index().set_index(['date','county'])[c+'_display']).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        
    d=dc.iloc[[0]]
    d['county']='d'
    d['case_14_cap']=4
    d['id']=99
        
    open(htmlipath+'panels/county2b_'+lang+'.json','w').write(json.dumps(list(d.T.to_dict().values())+list(dc.T.to_dict().values())))
    open(htmlipath+'panels/daily/county2b_'+latest+'_'+lang+'.json','w').write(json.dumps(list(d.T.to_dict().values())+list(dc.T.to_dict().values())))
    dms['lang']=lang
    dmss.append(dms)
    print(lang)

<ipython-input-66-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-66-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-66-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

HU


<ipython-input-66-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-66-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-66-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

RO


<ipython-input-66-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-66-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-66-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

EN


In [67]:
cases_counties2=open(htmlipath+'panels/cases_counties2.html','r').read()
cases_counties2=cases_counties2[:cases_counties2.find('daily/county2b_\' + \'2021-')+20]+latest+cases_counties2[cases_counties2.find('daily/county2b_\' + \'2021-')+30:]
open(htmlipath+'panels/cases_counties2.html','w').write(cases_counties2)

16605

Counties per day

In [68]:
dz=dh.set_index('EN').join(pd.DataFrame(pd.DataFrame(iso_counties,index=['county'])\
                                     .T.reset_index().set_index('county'))).reset_index()
dlz_data=pd.DataFrame(pd.DataFrame(dlz_counties).groupby('date').mean().stack()).reset_index()
dlz_data.columns=['date','index','value']
dz=dlz_data.set_index('index').join(dz.set_index('index')).dropna().set_index('date')
dz.index=pd.to_datetime(dz.index)
dz2=dz.copy()

In [69]:
# dz2=dz2.ffill()

In [70]:
dz=pd.DataFrame(dz.set_index(['county','value',1],append=True).stack()).reset_index()
dz.columns=['date','county','value','dump','lang','langtype']
dz=dz.set_index('date').drop('dump',axis=1)

In [71]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='counties'
push2influx(dz,measurement,field_columns,tag_columns)

Slicing counties from 2021-04-09 00:00:00+00:00 ...
Writing to counties ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Counties active

In [72]:
dz2=dz2.reset_index().set_index(['EN','HU','RO','county',1,'date'])\
    .unstack().diff(axis=1).stack().reset_index().set_index('date')

In [73]:
dz2=pd.DataFrame(dz2.set_index(['county','value',1],append=True).stack()).reset_index()
dz2.columns=['date','county','value','dump','lang','langtype']
dz2=dz2.set_index('date').drop('dump',axis=1)

In [74]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='counties2'
push2influx(dz2,measurement,field_columns,tag_columns)

Slicing counties2 from 2021-04-09 00:00:00+00:00 ...
Writing to counties2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


County intensity map

In [75]:
dmsr=pd.concat(dmss)[['case_14_cap','county','iso','lang']]
dmsr.index=pd.to_datetime(dmsr.index)

In [76]:
field_columns=['case_14_cap']
tag_columns=['county','iso','lang']
measurement='counties3'
push2influx(dmsr,measurement,field_columns,tag_columns)

Slicing counties3 from 2021-04-09 00:00:00+00:00 ...
Writing to counties3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Locality map

In [216]:
datagov1a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/bc19c354-644d-4a24-a26f-512129dbc70d/download/transparenta_vaccinare_martie_2021.xlsx')
datagov2a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d0b60b45-fb08-4980-a34c-8cbb4a43cad3/download/transparenta_martie_2021.xlsx',skiprows=1)
datagov1b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/a0eb9ff2-9b97-430c-b285-360cadb55307/download/vaccinare-covid19-grupe-risc-01-01.04.2021.xlsx')
datagov2b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d3280256-07cc-4f93-957a-9815085899be/download/transparenta_aprilie_2021.xlsx',skiprows=1)

In [78]:
datagov1=pd.concat([datagov1a,datagov1b])
datagov2=datagov2a.set_index(['Judet','UAT']).join(datagov2b.set_index(['Judet','UAT'])).reset_index()

In [79]:
del datagov1a
del datagov1b
del datagov2a
del datagov2b

In [80]:
uat=json.loads(open(htmlipath+'panels/uat_simplificat.geojson','r').read())

In [81]:
locs=[]
for f in uat['features']:
    locs.append(f['properties'])
locs=pd.DataFrame(locs)[['judet','uat','siruta']]

In [82]:
def judetconverter(c):
    return (' '.join([i.capitalize() for i in c.split(' ')])).replace('ţ','ț').replace('ş','ș').replace('năsăud','Năsăud').\
        replace('severin','Severin').replace('mare','Mare').\
        replace('Municipiul București','Bucharest').replace('București','Bucharest')

In [83]:
geo_iso=pd.DataFrame(iso_counties,index=['index']).T.reset_index().set_index('index').join(\
    pd.DataFrame(georoco,index=['geo_id']).T.reset_index().set_index('index')).reset_index()
geo_iso.columns=['en','iso2','geo_id']

In [84]:
locs['judet_norm']=[judetconverter(i) for i in locs['judet']]
datagov2['judet_norm']=[judetconverter(i) for i in datagov2['Judet']]

In [85]:
def uatconverter(judet,c):
    c=(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in c.split(' ')]))\
        .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
        .replace('î','â').replace(' De ',' de ')\
        .replace('Cuza Voda','Cuza Vodă')
    
    if (judet=='Dolj'):
        c=c.replace('Segarcea','Șegarcea')
    elif (judet=='Teleorman'):
        c=c.replace('Turnu Magurele','Turnu Măgurele')
    elif (judet=='Olt'):
        c=c.replace('Ipotesti','Ipotești')
    elif (judet=='Alba'):
        c=c.replace('Râmetea','Rimetea')
    elif (judet=='Constanța'):
        c=c.replace('44066','')
    elif (judet=='Ialomița'):
        c=c.replace('Radulești','Rădulești')
    elif (judet=='Argeș'):
        c=c.replace('Ciofringeni','Ciofrângeni')\
        .replace('Valea Mare Pravăț','Valea Mare-Pravăț')
    elif (judet=='Vâlcea'):
        c=c.replace('Păusești-Măglași','Păușești-Măglași')
    elif (judet=='Prahova'):
        c=c.replace('Cocorastii Colt','Cocorăștii Colț')
    elif (judet=='Bucharest'):
        c=c.replace('București Sectorul','Sector')
    elif (judet=='Gorj'):
        c=c.replace('Crușet','Crușeț')
    elif (judet=='Brăila'):
        c=c.replace('Racoviță','Racovița')\
        .replace('Gradiștea','Grădiștea')\
        .replace('Chișcani','Chiscani')
    elif (judet=='Cluj'):
        c=c.replace('Râșca','Rișca')
    elif (judet=='Botoșani'):
        c=c.replace('Răușeni','Răuseni')
    elif (judet=='Maramureș'):
        c=c.replace('Șisești','Șișești')
    elif (judet=='Brașov'):
        c=c.replace('Sambata de Sus','Sâmbăta de Sus')
    elif (judet=='Vaslui'):
        c=c.replace('Tătărăni','Tătărani')
    elif (judet=='Iași'):
        c=c.replace('Țigănăși','Țigănași')
    elif (judet=='Bistrița-Năsăud'):
        c=c.replace('Ilva Mica','Ilva Mică')
    elif (judet=='Bihor'):
        c=c.replace('Sănnicolau Romăn','Sânnicolau Român')
    elif (judet=='Mureș'):
        c=c.replace('Sarmașu','Sărmașu')
    return c

In [86]:
datagov2['uat_norm']=[(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in u.split(' ')]))\
                      .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
                      .replace(' De ',' de ').replace('Municipiul ','').replace('Oraș ','')\
                      for u in datagov2['UAT']]

In [87]:
uat_norms=[]
for judet in locs['judet_norm'].unique():
    for l in locs[locs['judet_norm']==judet]['uat'].unique():
        u=uatconverter(judet,str(l))
        if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
            u=u.replace(' ','-')
            if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
                print(judet,l,u)
                u=''
        uat_norms.append({'judet_norm':judet,'uat_norm':u,'uat':l})

Constanța 44066 
  


In [88]:
locs2=locs.set_index(['judet_norm','uat']).join(pd.DataFrame(uat_norms).set_index(['judet_norm','uat'])).reset_index()

In [89]:
datagov3=datagov2.set_index(['judet_norm','uat_norm']).join(locs2.set_index(['judet_norm','uat_norm']))
datagov3=datagov3.set_index(['judet','uat','siruta'],append=True).drop(['UAT','Judet'],axis=1)
datagov3.index=datagov3.index.reorder_levels([2,3,4,0,1])
# datagov3_dates=[pd.to_datetime(c) for c in datagov3.columns]
# datagov3_dates=[c for c in datagov3.columns]
datagov3_dates=[str(pd.to_datetime(c)) for c in datagov3.columns]

In [90]:
datagov3=datagov3.round(2)

In [91]:
del datagov1
del datagov2

In [92]:
def cat(r):
    if r<1: return '< 1 ‰'
    elif r<2: return '1-2 ‰'
    elif r<4: return '2-4 ‰'
    elif r<6: return '4-6 ‰'
    elif r<8: return '6-8 ‰'
    elif r<10: return '8-10 ‰'
    elif r<20: return '> 10-20 ‰'
    else: return '> 20 ‰'
def cat2(r):
    if r<1: return 0
    elif r<2: return 1
    elif r<4: return 2
    elif r<6: return 3
    elif r<8: return 4
    elif r<10: return 5
    elif r<20: return 6
    else: return 7

In [93]:
t=0
new_uat={'type':'FeatureCollection','features':[]}
new_uat_numbers=[]
new_uat_daily={'type':'FeatureCollection','features':[]}
for i in range(len(uat['features'])):
    dummy={}
    judet=uat['features'][i]['properties']['judet']
    if (judet!=''):
        u=uat['features'][i]['properties']['uat']
        siruta=uat['features'][i]['properties']['siruta']
        # dummy={'judet':judet,'uat':u,'siruta':siruta}
        if u in datagov3.loc[judet].index:
            d=datagov3.loc[judet].loc[u].loc[siruta]
            values=d.values[0]
            dummy['🌄']=d.index[0][0]
            dummy['🏠']=d.index[0][1]
            for t in range(len(values)):
                value=values[t]
                date=datagov3_dates[t]
                dummy['📈']=value
                dummy['🔴']=cat(value)
                dummy['⚫']=cat2(value)
                dummy['📆']=date
                feature={'type':'Feature','geometry':uat['features'][i]['geometry'],'properties':dummy.copy()}
                new_uat['features'].append(feature)
                new_uat_numbers.append(dummy.copy())
                if (t==len(datagov3_dates)-1):
                    new_uat_daily['features'].append(feature)

In [94]:
open(htmlipath+'panels/new_uat_numbers.json','w').write(json.dumps(new_uat_numbers))
open(htmlipath+'panels/new_uat.json','w').write(json.dumps(new_uat))

276910854

In [95]:
#Kepler.gl export
open(htmlipath+'panels/daily/new_uat'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/new_uat_daily.json','w').write(json.dumps(new_uat_daily))

7100456

In [96]:
del new_uat

In [97]:
#D3plus export

In [98]:
datagov4=datagov3.stack().reset_index()
datagov4.columns=['judet','uat','id','judet_norm','uat_norm','date','value']
datagov4['value']=np.round(datagov4['value'],2)
# datagov4=datagov4.dropna(subset=['judet'])
datagov4=datagov4.dropna(how='any')

In [99]:
open(htmlipath+'panels/daily/uat2b_'+date[:10]+'.json','w').write(json.dumps(list(datagov4.T.to_dict().values())))

19389725

In [100]:
# #HTML overwrite
# cases_localities=open(htmlipath+'panels/cases_localities.html','r').read()
# cases_localities=cases_localities[:cases_localities.find('daily/uat2b_\' + \'2021-')+17]+latest+cases_localities[cases_localities.find('daily/uat2b_\' + \'2021-')+27:]
# open(htmlipath+'panels/cases_localities.html','w').write(cases_localities)
# #### CHECK IF LATEST DATE IS ONE LESS than latest

Restrictions

In [101]:
df2=pd.DataFrame(new_uat_numbers)
df4=df2[['🌄', '🏠', '📈','📆']].set_index(['🌄', '🏠', '📆']).unstack()['📈'].sort_index(axis=1)

In [102]:
def restrictie(x,w):
    if x>7.5: return 4
    elif x>7:
        if w>7.5: return 3
        else: return 2
    elif x>4: return 2
    elif x>3.5:
        if w>4: return 1
        else: return 0
    else:
        return 0
    
def restrictie2(x,w):
    if x>7.5: return '7.5+'
    elif x>7:
        if w>7.5: return '7.5-'
        else: return '4+'
    elif x>4: return '4+'
    elif x>3.5:
        if w>4: return '4-'
        else: return '0'
    else:
        return '0'

In [103]:
dr=[]
for i in df4.T.iteritems():
    v=i[1]
    #print(i[0])
    for j in range(len(v)):
        t=v.index[j]
        x=v[j]
        y=v[j-min(14,j):j].values
        w=x
        if len(y):
            w=max(y)
        r=restrictie(x,w)
        #print(j,t,x,w,r)
        dr.append({'🌄':i[0][0], '🏠':i[0][1], '🚦':r,'📆':t})

In [104]:
dp=pd.DataFrame(dr).set_index(['🌄', '🏠','📆']).astype(float)

In [105]:
for i in new_uat_daily['features']:
    i['properties']['🚦']=dp.loc[i['properties']['🌄']].loc[i['properties']['🏠']].loc[i['properties']['📆']]['🚦']

In [106]:
open(htmlipath+'panels/new_uat_daily2.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/daily/new_uat_daily2_'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))

7167320

In [107]:
#HTML overwrite
uat_html=open(htmlipath+'panels/daily/mapbox-'+date[:10]+'.html','r').read()
uat_html=uat_html.replace(date[5:10],date[5:8]+today0).replace(date[5:10],date[5:8]+today0)
uat_html=uat_html[:uat_html.find('.json')-2]+\
    str(pd.to_datetime(now)+pd.to_timedelta('-1d'))[8:10]+\
    uat_html[uat_html.find('.json'):]
open(htmlipath+'panels/daily/mapbox-'+date[:8]+today0+'.html','w').write(uat_html)

18658

Upload to `uat.json` to mapbox

In [274]:
#Mapbox colors:
# 239e69
# f5d232
# c52b69
# 850237

In [108]:
#Add szekelydata GitHub update
#Add mapbox direct upload

In [109]:
#Sanity check
datagov3.loc['Cluj'].loc['Cluj-Napoca'].T.tail(3)

siruta,54975
judet_norm,Cluj
uat_norm,Cluj-Napoca
2021-04-06,8.01
2021-04-07,7.84
2021-04-08,7.67


Mapbox upload

In [110]:
import requests

In [111]:
mapbox_api_key=open('auth/mapbox.txt','r').read()
url='https://api.mapbox.com/styles/v1/<my_user>/<my_style>?access_token='+mapbox_api_key
r=requests.get(url)
mbstyle=json.loads(r.content)

In [112]:
mbstyle['name']='<my_style_name>'
# mbstyle['modified']=str(pd.to_datetime('now')).replace(' ','T')[:23]+'Z'
mbstyle.pop('modified');
mbstyle.pop('created');
mbstyle['layers'][0]['paint']['fill-color'][4]='#ffee00'
mbstyle['layers'][0]['paint']['fill-color'][10]='#ffffff'
mbstyle['layers'][0]['paint']['fill-color'][2][1]='⚫'

In [113]:
head = {"Content-Type":"application/json"}
r = requests.patch(url, json.dumps(mbstyle), headers=head)
r

<Response [200]>

In [114]:
# json.loads(r.content)

Wait 5 min

Push to Grafana

In [115]:
nevuto={'0':'á','1':'é','3':'á','2':'á','4':'é','5':'é','6':'á','7':'é','8':'á','9':'é'}
for lang in languages:
    response = requests.get(grafana+'api/dashboards/uid/'+lang.lower(), headers=headers)
    model=json.loads(response.content)['dashboard']
    for i in model['panels']:
        if lang=='HU':
            #print(i['id'],i['type'],i['title'])
            if i['id'] in (179, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find('n,')-4].strip()+' '+today+'-'+\
                    nevuto[now[-1]]+i['description'][i['description'].find('n,'):\
                    i['description'].find(':')-2]+hour+\
                    i['description'][i['description'].find(':')+3:]
                i['title']=i['title'][:i['title'].rfind(' ')]+' '+today
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='EN':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' on '+today+' '+ i['description'][i['description'].find('on ')+5:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+3:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='RO':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' în data de '+today+' '+ i['description'][i['description'].find('în data de ')+13:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+3:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('<b>')+3]+today0+\
                i['content'][i['content'].rfind('<b>')+5:]
            
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })       
    print(r,r.content)
    open(lang+'.json','w').write(json.dumps(model))    
    
    model_light=json.dumps(model).replace('dark','light',9999).replace('lightGray','#52545C',9999)\
        .replace('#d3d3d3','#52545c',9999)\
        .replace('csaladen.es/favicon.ico" style="','csaladen.es/favicon.ico" style="filter: brightness(0.3);',9999)
    for color in colors_to_darken:
        model_light=model_light.replace(color,adjust_lightness(color,0.8),9999)
        model_light=model_light.replace(color.lower(),adjust_lightness(color,0.8),9999)
    model=json.loads(model_light)
    model['title']=titles[lang]+' Light'
    model['uid']=uids_light[lang]
    model['id']=iids_light[lang]
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })    
    print(r,r.content)
    open(lang+'-light.json','w').write(json.dumps(model))   

<Response [200]> b'{"id":2,"slug":"magyar","status":"success","uid":"hu","url":"/d/hu/magyar","version":292}'
<Response [200]> b'{"id":5,"slug":"magyar-light","status":"success","uid":"hu-light","url":"/d/hu-light/magyar-light","version":128}'
<Response [200]> b'{"id":3,"slug":"romana","status":"success","uid":"ro","url":"/d/ro/romana","version":197}'
<Response [200]> b'{"id":6,"slug":"romana-light","status":"success","uid":"ro-light","url":"/d/ro-light/romana-light","version":122}'
<Response [200]> b'{"id":4,"slug":"english","status":"success","uid":"en","url":"/d/en/english","version":161}'
<Response [200]> b'{"id":7,"slug":"english-light","status":"success","uid":"en-light","url":"/d/en-light/english-light","version":123}'
